In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
X_train = X_train / 255.0
X_val = X_val / 255.0

*Normal*

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
model.fit(X_train, Y_train, epochs=20, batch_size=20, validation_data=(X_val, Y_val))

Epoch 1/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.8301 - loss: 0.4843 - val_accuracy: 0.6984 - val_loss: 1.0186
Epoch 2/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8393 - loss: 0.4498 - val_accuracy: 0.7043 - val_loss: 1.0351
Epoch 3/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8603 - loss: 0.4039 - val_accuracy: 0.6937 - val_loss: 1.0710
Epoch 4/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8704 - loss: 0.3658 - val_accuracy: 0.6950 - val_loss: 1.1066
Epoch 5/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8797 - loss: 0.3335 - val_accuracy: 0.6859 - val_loss: 1.1756
Epoch 6/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.8930 - loss: 0.2998 - val_accuracy: 0.6945 - val_loss: 1.2446
Epoch 7/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9028 - loss: 0.2769 - val_accuracy: 0.6932 - val_loss: 1.3037
Epoch 8/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9099 - loss: 0

In [69]:
model.evaluate(x_test / 255.0, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6651 - loss: 2.1798


[2.2744457721710205, 0.6617000102996826]

In [70]:
model.save('cifar10_model_1.h5')

*Hyper Parameter tuning*

In [20]:
def build_model(hp):
    model1 = Sequential()

    # Hyperparameter for the number of filters in Conv2D layers
    filters_1 = hp.Int('filters_1', min_value=32, max_value=128, step=32)
    filters_2 = hp.Int('filters_2', min_value=64, max_value=256, step=64)
    filters_3 = hp.Int('filters_3', min_value=64, max_value=256, step=64)

    # Hyperparameter for kernel size
    kernel_size_1 = hp.Choice('kernel_size_1', values=[3, 5])
    kernel_size_2 = hp.Choice('kernel_size_2', values=[3, 5])
    kernel_size_3 = hp.Choice('kernel_size_3', values=[3, 5])

    # Hyperparameter for activation function
    activation = hp.Choice('activation', values=['relu', 'tanh', 'elu', 'swish'])

    # First Conv2D layer
    model1.add(layers.Conv2D(filters=filters_1, kernel_size=(kernel_size_1, kernel_size_1), activation=activation, input_shape=(32, 32, 3)))
    model1.add(layers.MaxPooling2D((2, 2)))

    # Second Conv2D layer
    model1.add(layers.Conv2D(filters=filters_2, kernel_size=(kernel_size_2, kernel_size_2), activation=activation))
    model1.add(layers.MaxPooling2D((2, 2)))

    # Third Conv2D layer
    model1.add(layers.Conv2D(filters=filters_3, kernel_size=(kernel_size_3, kernel_size_3), activation=activation))

    # Flatten the output and add Dense layers
    model1.add(layers.Flatten())

    # Hyperparameter for Dense layer units
    dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=32)
    model1.add(layers.Dense(dense_units, activation=activation))

    # Output layer with 10 units (for CIFAR-10)
    model1.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model1

In [21]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',  # We want to optimize for validation accuracy
    max_epochs=10,  # Number of epochs to run for each trial
    factor=3,  # Controls the resource allocation between trials
    directory='hyperband_results',  # Directory to store results
    project_name='cifar10_tuning'
)

/Users/pravinpb/pycode/MCW/Assignment1/cifar10/cifar/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [24]:
tuner.search(x_train, y_train, epochs=10, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Trial 26 Complete [00h 09m 36s]
val_accuracy: 0.121799997985363

Best val_accuracy So Far: 0.19910000264644623
Total elapsed time: 04h 15m 00s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
96                |32                |filters_1
256               |192               |filters_2
192               |64                |filters_3
5                 |5                 |kernel_size_1
3                 |3                 |kernel_size_2
5                 |5                 |kernel_size_3
relu              |tanh              |activation
128               |32                |dense_units
10                |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 98s 62ms/step - accuracy: 0.3561 - loss: 3.6246 - val_accuracy: 0.1440 - val_loss: 2.3709
Epoch 2/10
1258/1563 ━━━━━━━━━━━━

KeyboardInterrupt: 

In [12]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [13]:
best_model.fit(x_train, y_train, epochs=10, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.2762 - loss: 1.9998 - val_accuracy: 0.3710 - val_loss: 1.7610
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.3862 - loss: 1.7129 - val_accuracy: 0.3951 - val_loss: 1.6995
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4212 - loss: 1.6113 - val_accuracy: 0.4396 - val_loss: 1.5883
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4522 - loss: 1.5468 - val_accuracy: 0.4420 - val_loss: 1.5616
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4760 - loss: 1.4907 - val_accuracy: 0.4326 - val_loss: 1.6009
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4814 - loss: 1.4532 - val_accuracy: 0.4658 - val_loss: 1.5174
Epoch 7/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4996 - loss: 1.4110 - val_accuracy: 0.4725 - val_loss: 1.5046
Epoch 8/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5098 - loss: 1.3862 - 